In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [115]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
cuda.use('gpu0')

## Read data

In [123]:
big_path = "data/movielens/big/"
#path = big_path
path = "data/movielens/small/"
batch_size=64

In [124]:
ratings = pd.read_csv(path+'ratings.csv')
movie_names = pd.read_csv(big_path+'movies.csv').set_index('movieId')['title'].to_dict()


In [125]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()
n_users, n_movies

(671, 9066)

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205
5,0,5,2.0,1260759151
6,0,6,2.0,1260759187
7,0,7,2.0,1260759148
8,0,8,3.5,1260759125
9,0,9,2.0,1260759131


## Create test/validation set

In [126]:
n_factors = 50
np.random.seed = 42

In [127]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [164]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [165]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [166]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
embedding_model = Model([user_in, movie_in], x)
embedding_model.compile(Adam(0.001), loss='mse')

In [170]:
embedding_model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80096 samples, validate on 19908 samples
Epoch 1/3
80096/80096 [==============================] - 6s - loss: 1.7661 - val_loss: 1.1763
Epoch 2/3
80096/80096 [==============================] - 6s - loss: 1.6862 - val_loss: 1.1342
Epoch 3/3
80096/80096 [==============================] - 6s - loss: 1.6077 - val_loss: 1.1027


In [174]:
embedding_model.optimizer.lr=0.001

In [176]:
embedding_model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80096 samples, validate on 19908 samples
Epoch 1/5
80096/80096 [==============================] - 6s - loss: 1.0223 - val_loss: 0.9034
Epoch 2/5
80096/80096 [==============================] - 6s - loss: 0.9723 - val_loss: 0.8873
Epoch 3/5
80096/80096 [==============================] - 6s - loss: 0.9254 - val_loss: 0.8722
Epoch 4/5
80096/80096 [==============================] - 6s - loss: 0.8811 - val_loss: 0.8596
Epoch 5/5
80096/80096 [==============================] - 6s - loss: 0.8410 - val_loss: 0.8448


In [146]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')
nn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 1, 50)         33550       user_in[0][0]                    
____________________________________________________________________________________________________
embedding_6 (Embedding)          (None, 1, 50)         453300      movie_in[0][0]                   
___________________________________________________________________________________________

In [147]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=8, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80096 samples, validate on 19908 samples
Epoch 1/8
80096/80096 [==============================] - 7s - loss: 1.8071 - val_loss: 0.8151
Epoch 2/8
80096/80096 [==============================] - 7s - loss: 1.0697 - val_loss: 0.8054
Epoch 3/8
80096/80096 [==============================] - 7s - loss: 1.0018 - val_loss: 0.7937
Epoch 4/8
80096/80096 [==============================] - 7s - loss: 0.9446 - val_loss: 0.7945
Epoch 5/8
80096/80096 [==============================] - 7s - loss: 0.8933 - val_loss: 0.7873
Epoch 6/8
80096/80096 [==============================] - 7s - loss: 0.8548 - val_loss: 0.7853
Epoch 7/8
80096/80096 [==============================] - 7s - loss: 0.8253 - val_loss: 0.7854
Epoch 8/8
80096/80096 [==============================] - 7s - loss: 0.8053 - val_loss: 0.7816


In [148]:
nn.predict([val.userId[:30], val.movieId[:30]])

array([[ 2.8733],
       [ 3.7771],
       [ 4.0605],
       [ 3.662 ],
       [ 3.1894],
       [ 2.9301],
       [ 3.5793],
       [ 3.8821],
       [ 3.0451],
       [ 3.3847],
       [ 3.1447],
       [ 3.3436],
       [ 4.0229],
       [ 2.644 ],
       [ 3.0551],
       [ 3.289 ],
       [ 3.8455],
       [ 3.3765],
       [ 3.658 ],
       [ 3.5154],
       [ 3.6308],
       [ 3.296 ],
       [ 3.8408],
       [ 3.3577],
       [ 3.8793],
       [ 3.8509],
       [ 3.1666],
       [ 4.3362],
       [ 4.2525],
       [ 4.698 ]], dtype=float32)

In [149]:
val.rating[:30]

1      3.0
23     4.0
24     4.0
29     5.0
34     3.0
47     4.0
55     4.0
57     3.0
60     3.0
64     3.0
80     4.0
82     3.0
100    4.5
102    2.5
107    3.0
110    3.5
113    5.0
119    4.5
127    3.5
134    3.0
136    3.0
139    3.5
142    3.5
143    4.5
144    4.5
145    3.0
152    3.0
161    5.0
171    5.0
174    5.0
Name: rating, dtype: float64

## Analyze embeddings

In [150]:
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:2000]
topMovies = np.array(topMovies.index)
topMovies[:30]

array([ 57,  49,  99,  92, 143,  72, 402, 417,  79,  89, 179,  27, 197, 505, 180, 121, 432, 106,
        88,  69, 122,  90,  23,  24,  59, 157, 480,  29, 146,  91])

Getting the movie embedding by assosiating the movie_in output with the movie embedding as output. And predicting the output for the topMovies

In [151]:
get_movie_emb = Model(movie_in, mb)
movie_emb = np.squeeze(get_movie_emb.predict([topMovies]))
movie_emb.shape

(2000, 50)

Reducing the 50 embeddings to 3-dimensional space

In [152]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
movie_pca = pca.fit(movie_emb.T).components_

In [153]:
movie_pca

array([[ 0.0367,  0.0516,  0.0618, ...,  0.037 , -0.0177,  0.0075],
       [-0.0071, -0.0077, -0.041 , ...,  0.0085,  0.0238, -0.0208],
       [-0.0179, -0.0472, -0.0356, ..., -0.0026, -0.0065,  0.0565]], dtype=float32)

Get first embedding

In [154]:
def get_movie(m_id):
    if m_id in movie_names: 
         return movie_names[m_id]
    else:
        return "no movie found {}".format(m_id)
    
def get_embedding(index = 0, number = 10):
    fac = movie_pca[index]
    f_top = zip(fac, topMovies)
    movie_comp = map(lambda (f, i): [f, get_movie(i)], f_top)
    top = sorted(movie_comp, key=itemgetter(0), reverse=True)[:number]
    bottom = sorted(movie_comp, key=itemgetter(0), reverse=False)[:number]
    return (top, bottom)

In [155]:
top, bottom = get_embedding(0)


In [156]:
top

[[0.06184097, 'Heidi Fleiss: Hollywood Madam (1995)'],
 [0.059043795, 'Canadian Bacon (1995)'],
 [0.057739392, '8MM (1999)'],
 [0.057604756, "Margaret's Museum (1995)"],
 [0.057032388, 'North (1994)'],
 [0.05616419, 'Faster Pussycat! Kill! Kill! (1965)'],
 [0.054973863, 'Congo (1995)'],
 [0.053878572, 'Grass Harp, The (1995)'],
 [0.05359941, 'Fluke (1995)'],
 [0.05329667, 'Mad Love (1995)']]

In [157]:
bottom

[[-0.053857327, 'Lost Horizon (1937)'],
 [-0.053489275, 'Manhunter (1986)'],
 [-0.052709322, 'no movie found 530'],
 [-0.052116994, 'Saboteur (1942)'],
 [-0.05175006, 'Striking Distance (1993)'],
 [-0.051420242, 'no movie found 641'],
 [-0.050937317, 'Repulsion (1965)'],
 [-0.050737116, 'World of Apu, The (Apur Sansar) (1959)'],
 [-0.0507369, 'no movie found 2954'],
 [-0.050330609, 'no movie found 860']]

In [158]:
top, bottom = get_embedding(1)

In [159]:
top

[[0.086403452, 'Messenger: The Story of Joan of Arc, The (1999)'],
 [0.080959246, 'Beyond Bedlam (1993)'],
 [0.071455918, 'My Man Godfrey (1936)'],
 [0.06994348, 'no movie found 1121'],
 [0.06811294, 'Miracle on 34th Street (1994)'],
 [0.065104663, 'True Lies (1994)'],
 [0.064722024, 'Popeye (1980)'],
 [0.063206099, 'Tom and Huck (1995)'],
 [0.062648088, "Logan's Run (1976)"],
 [0.06117494, 'Dumb & Dumber (Dumb and Dumber) (1994)']]

In [160]:
bottom

[[-0.063700005, 'Midnight in the Garden of Good and Evil (1997)'],
 [-0.056820404, 'Seventh Heaven (Septi\xc3\xa8me ciel, Le) (1997)'],
 [-0.056591112, 'Deceiver (1997)'],
 [-0.053776048, 'Trouble with Harry, The (1955)'],
 [-0.053616233, 'Meteor (1979)'],
 [-0.052511379, 'no movie found 1309'],
 [-0.05233426, 'Terminator, The (1984)'],
 [-0.050409358, 'no movie found 1520'],
 [-0.049084403, 'To Kill a Mockingbird (1962)'],
 [-0.049019802, 'Fresh (1994)']]

In [161]:
top, bottom = get_embedding(2)

In [162]:
top

[[0.079137012, 'Father of the Bride Part II (1995)'],
 [0.070415847, 'Baby-Sitters Club, The (1995)'],
 [0.069486454, 'no movie found 4582'],
 [0.069242291, 'Made in America (1993)'],
 [0.068516225, 'Pie in the Sky (1996)'],
 [0.064876236, 'Cutthroat Island (1995)'],
 [0.063664012, 'Beverly Hills Ninja (1997)'],
 [0.061730031, 'Favor, The (1994)'],
 [0.061348364, 'Blade Runner (1982)'],
 [0.058032352, 'Picnic (1955)']]

In [163]:
bottom

[[-0.092232063, 'Threesome (1994)'],
 [-0.071114406, 'no movie found 1315'],
 [-0.070515566, 'Primal Fear (1996)'],
 [-0.0695673, 'Getting Even with Dad (1994)'],
 [-0.067315705,
  'Halloween: The Curse of Michael Myers (Halloween 6: The Curse of Michael Myers) (1995)'],
 [-0.066656969, 'Wolf (1994)'],
 [-0.065920226, 'no movie found 385'],
 [-0.065919131, 'Ladybird Ladybird (1994)'],
 [-0.06477464, 'Snake Eyes (1998)'],
 [-0.06209233, 'Tom and Huck (1995)']]

array([[-0.0329, -0.0458, -0.0558, ..., -0.0278,  0.0262,  0.0072],
       [ 0.0066,  0.0049, -0.0134, ...,  0.0132,  0.0107, -0.017 ],
       [-0.0628, -0.0063,  0.0045, ..., -0.0357,  0.0205, -0.0276]], dtype=float32)